In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
import optuna
from optuna.samplers import TPESampler

In [4]:
# Load dataset
df = pd.read_excel(r"C:\Users\jazil\OneDrive\pep-ds\sleep-proj\Balanced_Sleep_Quality_Data.xlsx")

# Prepare features and target
X = df.drop(columns=['Person ID', 'Quality of Sleep'])
y = df['Quality of Sleep']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocess data upfront: encode categorical and scale numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols),
        ('num', StandardScaler(), numerical_cols)
    ],
    remainder='passthrough'
)

In [6]:
# Fit and transform the entire dataset once
X_processed = preprocessor.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Define the Optuna objective function for hyperparameter tuning
def objective(trial):
    # Suggest hyperparameters for Random Forest
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 100, 300)
    rf_max_depth = trial.suggest_int('rf_max_depth', 5, 15)

    # Suggest hyperparameters for Neural Network
    nn_units1 = trial.suggest_int('nn_units1', 50, 200)
    nn_units2 = trial.suggest_int('nn_units2', 25, 100)
    nn_alpha = trial.suggest_float('nn_alpha', 1e-5, 1e-3, log=True)

    # Define base learners
    rf_model = RandomForestRegressor(
        n_estimators=rf_n_estimators,
        max_depth=rf_max_depth,
        random_state=42,
        n_jobs=-1
    )

    nn_model = MLPRegressor(
        hidden_layer_sizes=(nn_units1, nn_units2),
        activation='relu',
        solver='adam',
        random_state=42,
        max_iter=500,
        alpha=nn_alpha,
        early_stopping=True
    )
     # Define meta-learner
    meta_model = RandomForestRegressor(random_state=42, n_jobs=-1)
    stacking_regressor = StackingRegressor(
        estimators=[
            ('rf', rf_model),
            ('nn', nn_model),
        ],
        final_estimator=meta_model,
        cv=5,
        passthrough=False
    )

    # Train model
    stacking_regressor.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = stacking_regressor.predict(X_test)
    return r2_score(y_test, y_pred)

In [7]:
# Create Optuna study and optimize
sampler = TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=50)

[I 2025-06-09 16:00:53,643] A new study created in memory with name: no-name-90d3e2f4-8fc5-4a08-bac5-0477d83f14f1
[I 2025-06-09 16:01:04,455] Trial 0 finished with value: 0.8288140343239875 and parameters: {'rf_n_estimators': 175, 'rf_max_depth': 15, 'nn_units1': 160, 'nn_units2': 70, 'nn_alpha': 2.0513382630874486e-05}. Best is trial 0 with value: 0.8288140343239875.
[I 2025-06-09 16:01:14,510] Trial 1 finished with value: 0.8704909379546529 and parameters: {'rf_n_estimators': 131, 'rf_max_depth': 5, 'nn_units1': 180, 'nn_units2': 70, 'nn_alpha': 0.0002607024758370766}. Best is trial 1 with value: 0.8704909379546529.
[I 2025-06-09 16:01:24,991] Trial 2 finished with value: 0.8759915953609922 and parameters: {'rf_n_estimators': 104, 'rf_max_depth': 15, 'nn_units1': 175, 'nn_units2': 41, 'nn_alpha': 2.3102018878452926e-05}. Best is trial 2 with value: 0.8759915953609922.
[I 2025-06-09 16:01:32,608] Trial 3 finished with value: 0.8815891880052097 and parameters: {'rf_n_estimators': 136, 

In [8]:
# Output best trial results
print("Best trial:")
trial = study.best_trial
print(f"  R2 Score: {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  R2 Score: 0.9076
  Params:
    rf_n_estimators: 262
    rf_max_depth: 12
    nn_units1: 59
    nn_units2: 29
    nn_alpha: 1.6023703509843143e-05


In [9]:
# Train final model with best hyperparameters
best_rf_n_estimators = trial.params['rf_n_estimators']
best_rf_max_depth = trial.params['rf_max_depth']
best_nn_units1 = trial.params['nn_units1']
best_nn_units2 = trial.params['nn_units2']
best_nn_alpha = trial.params['nn_alpha']

rf_model = RandomForestRegressor(
    n_estimators=best_rf_n_estimators,
    max_depth=best_rf_max_depth,
    random_state=42,
    n_jobs=-1
)

nn_model = MLPRegressor(
    hidden_layer_sizes=(best_nn_units1, best_nn_units2),
    activation='relu',
    solver='adam',
    random_state=42,
    max_iter=500,
    alpha=best_nn_alpha,
    early_stopping=True
)

meta_model = RandomForestRegressor(random_state=42, n_jobs=-1)

stacking_regressor = StackingRegressor(
    estimators=[
        ('rf', rf_model),
        ('nn', nn_model),
    ],
    final_estimator=meta_model,
    cv=5,
    passthrough=False
)


In [10]:
# Fit final model
stacking_regressor.fit(X_train, y_train)

# Final evaluation on test set
y_pred = stacking_regressor.predict(X_test)
print(f"Final R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"Final MSE: {mean_squared_error(y_test, y_pred):.4f}")

Final R2 Score: 0.9076
Final MSE: 0.3784


In [11]:
# After training in your notebook
import joblib
joblib.dump(stacking_regressor, 'stacking_regressor.pkl')
joblib.dump(preprocessor, 'preprocessor.pkl')


['preprocessor.pkl']

In [12]:
print(X.columns.tolist())  # feature column names
print(X.dtypes)            # feature data types


['Gender', 'Age', 'Occupation', 'Sleep Duration', 'Physical Activity Level', 'Stress Level', 'BMI Category', 'Blood Pressure', 'Heart Rate', 'Daily Steps']
Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Physical Activity Level      int64
Stress Level               float64
BMI Category                object
Blood Pressure              object
Heart Rate                   int64
Daily Steps                  int64
dtype: object
